# Creating Message Passing Networks
**在这里，使用PYG框架实现了GCN和DGCNN**

PYG 提供了MessagePassing的基类，会自己进行信息传递，用户需要定义phi、gamma函数和汇聚模式（sum、mean、max）

```python
MessagePassing(aggr="add", flow="source_to_target", node_dim=-2)
```

以下方法让我们能够进行网络的创建
1. MessagePassing(aggr="add",flow=source_to_target",node_dim=-2)

  node_dim : 哪个维度的数据被传播？

2. MessagePassing.propagate(edge_index,size=None,**kwargs)

进行消息传播的初始调用，将节点的信息和边的信息收集起来，然后进行节点向量的更新

3. MessagePassing.message(...) 构建节点i的信息，参数是通过propagate的参数。

4. MessagePassing.update(aggr_out,...),更新所有的节点的嵌入向量

# IMPLEMETING THE GCN LAYER


In [ ]:
import torch as t
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import  add_self_loops,degree
import torch.nn as nn

class GCNConv(MessagePassing):
    def __init__(self, in_channels,out_channels):
        super().__init__(aggr="add")
        self.lin=t.nn.Linear(in_channels,out_channels)

    def forward(self, x:t.Tensor, edge_index: t.Tensor):
        # x.shape = [N,in_channels]
        # edge_index.shape = [2,E] where E denotes the number of edges
        # 
        # Step 1: Add self-loops to the adjacency matrix
        edge_index,_=add_self_loops(edge_index,num_nodes=x.size(0))

        # Step 2 : linearly transform node feature matrix
        x= self.lin(x)

        # Step 3 : Compute normalization
        row,col=edge_index
        deg=degree(col,x.size(0),dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt==float('inf')]=0
        norm = deg_inv_sqrt[row]*deg_inv_sqrt[col]

        # Step 4-5 : Start propagating messages.
        return self.propagate(edge_index,x=x,norm=norm)

    def message(self, x_j: t.Tensor, norm: t.Tensor) -> t.Tensor:
        # x_j.shape = [e,out_channels]
        # Step 4 : normalize node features
        return norm.view(-1,1)*x_j


In [ ]:
class EdgeConv(MessagePassing):
    def __init__(self, in_channels,out_channels):
        super().__init__(aggr="max")
        self.mlp = nn.Sequential(
            nn.Linear(2*in_channels,out_channels),
            nn.ReLU(),
            nn.Linear(out_channels,out_channels)
        )

    def forward(self, x: t.Tensor, edge_index: t.Tensor):
        # x.shape = [n, in_channels]
        # edge_index = [2, E]
        return self.propagate(edge_index,x)

    def message(self, x_i: t.Tensor,x_j: t.Tensor) -> t.Tensor:
        # x_i has shape [E, in_channels]
        # x_j has shape [E, in_channels]
        tmp = t.cat([x_i,x_j-x_i],dim=1) # shape = [E,2*in_channels]
        return self.mlp(tmp)

摆烂不翻译：The edge convolution is actually a dynamic convolution, which recomputes the graph for each layer using nearest neighbors in the feature space.

Luckily, PyG comes with a GPU accelerated batch-wise k-NN graph generation method named torch_geometric.nn.pool.knn_graph():

In [ ]:
from torch_geometric.nn import knn_graph

class DynamicEdgeConv(EdgeConv):
    def __init__(self, in_channels, out_channels,k=6):
        super().__init__(in_channels, out_channels)
        self.k=k
    def forward(self,x,batch=None):
        edge_index = knn_graph(x,self.k,batch,loop=False,flow=self.flow)
        return super().forward(x,edge_index)